Pobieranie wiadomości z polskieradio24.pl
Zapis do pliku ./data/pr24_{timestamp}.txt

In [1]:
# ! pip install bs4
# ! pip install requests
# ! pip install tqdm
# ! pip install pandas
# ! pip install pyarrow
# ! pip install qgrid
# ! pip install blaze
# ! pip install ipynbname

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm, trange
from datetime import datetime, timezone
from common.webarticle import WebArticle
from common.pl_helper import PolishLocaleHelper
import pandas as pd
import logging
import ipynbname

In [3]:
# settings:
portal_name = 'pr24'
website = 'https://polskieradio24.pl/321/6713'
pagination_attributes = '/Strona/'
no_articles_to_scrap = 300
data_dir = '../data'
logging.basicConfig(
    filename=f'../logs/{ipynbname.name()}.ipynb.log',
    encoding='utf-8',
    level=logging.DEBUG
)
ignored_paragraph_starts = ['POSŁUCHAJ', 'Zobacz także:']

Sprawdźmy plik robots.txt, w czasie pisania tego kodu pozwalał na pobieranie plików z portalu pr24 z kategorii Polska

In [4]:
resp = requests.get('https://polskieradio24.pl/robots.txt')
print(resp.text)

User-agent: *
Disallow: /cms/
Disallow: /portal/player/PlayerGG.aspx
Disallow: /loguj/
Disallow: /8,Dwojka/41,Kontakt
Disallow: /9/6500
Disallow: /9,Trojka/6291,Ku-Niepodleglej
Disallow: /Portal/AddUser.aspx
Disallow: /82/1788/Artykul/872890%2CW-Polsce-nie-widac-uczciwych-ludzi
Disallow: /7/6295
Disallow: /Polskie-Radio-24/Tag99941
Disallow: /43/581/ArtykulTime/2246911,VIII-etap-Rajdu-Dakar-2019-San-Juan-de-Marcona-Pisco
Disallow: /images/
Disallow: /*.file$
Disallow: /*.mp3$
Disallow: /*.jpg$
Disallow: /43/581/ArtykulTime/2247561,IX-etap-Rajdu-Dakar-2019-Pisco
Disallow: /43/581/Artykul/2234438,PS-w-skokach-narciarskich-w-Engelbergu
Disallow: /5/1223/Artykul/2267647
Disallow: /13/53/Artykul/2297139,Wielkie-spiewanie-i-granie-Moniuszki-w-200-rocznice-jego-urodzin
Disallow: /53/Artykul/2297139,Wielkie-spiewanie-i-granie-Moniuszki-w-200-rocznice-jego-urodzin
Disallow: /Artykul/2297139,Wielkie-spiewanie-i-granie-Moniuszki-w-200-rocznice-jego-urodzin
Disallow: /10/
Disallow: /10,Czworka/
Di

In [5]:
def check_if_ignored_paragraph(text, ignored_paragraphs, operation='startswith'):

    for ignored_paragraph in ignored_paragraphs:
        if operation == 'startswith':
            if text.startswith(ignored_paragraph):
                return True
        else:
            logging.debug(f'check_if_ignored: operation {operation} not implemented')
    return False


In [6]:
def scrap_article_text_pr24(link):
    logging.info('scrap_article_text_pr24:')
    logging.info(f'article link to scrap: {link}')

    page = requests.get(link)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return '', ''

    soup = BeautifulSoup(page.content, 'html.parser')
    paragraphs = soup.find('div', class_='content').find_all('p')

    text = ''
    for paragraph in paragraphs[:-1]:
        paragraph_text = paragraph.get_text(strip=True)
        if check_if_ignored_paragraph(paragraph_text, ignored_paragraph_starts):
            continue
        text += paragraph_text + ' '

    time_string = soup.find('div', class_='article-time').find('span', class_='time').get_text().strip()
    when_published = datetime.strptime(time_string, "%d.%m.%Y %H:%M")

    title_int = soup.find('h1', class_='title').get_text().strip()

    lead_text_int = soup.find('div', class_='desc').get_text().strip()
    author = ''
    author_candidate = paragraphs[-1].get_text(strip=True)
    if len(author_candidate) <= 50:
        author = author_candidate
    else:
        text += author_candidate

    source = soup.find('div', class_='article-source').find('span').get('class')[1]

    return text, when_published, title_int, lead_text_int, author, source

In [7]:
def scrap_articles_pr24(website):
    logging.info('scrap_articles_pr24:')
    logging.info(f'page with articles to scrap: {website}')

    result = list()
    page = requests.get(website)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return result

    soup = BeautifulSoup(page.content, 'html.parser')
    page_content = soup.select_one('div.articles-list-with-date').find_all('div',class_='articles-grid__col articles-grid__col_12-large')

    print(f'articles to scrap: {len(page_content)}')

    for article in page_content:

        logging.info(f"article scrapping started: {article}")
        title = article.find('h2',class_='article__title article__title_large').get_text().strip()
        lead_text = article.find('span',class_='article__lead').get_text().strip()
        link = article.find_all('a', href=True, class_='main-link')[0]['href']
        webart = WebArticle(title, lead_text, link, *scrap_article_text_pr24(link),portal_name)
        logging.info(f'webarticle scrapped: {webart}')
        result.append(webart)

    return result

In [8]:
no_scrapped_articles = 0
page_no=1
scrapped_articles = list()

logging.info(f'no_articles_to_scrap: {no_articles_to_scrap}')

pbar = tqdm(total=no_articles_to_scrap)
while no_scrapped_articles < no_articles_to_scrap:
    logging.info('next page: '+website+pagination_attributes+str(page_no))
    new_articles = scrap_articles_pr24(website+pagination_attributes+str(page_no))
    page_size = len(new_articles)
    logging.info(f'new_articles: {page_size}')
    if no_scrapped_articles + page_size > no_articles_to_scrap:
        page_size = no_articles_to_scrap - no_scrapped_articles
    scrapped_articles.extend(new_articles[:page_size])
    logging.info(f'scrapped new articles: {page_size}')
    no_scrapped_articles += page_size
    pbar.update(page_size)
    logging.info(f'no_scrapped_articles (total): {no_scrapped_articles}')
    page_no += 1
    if len(scrapped_articles) <= 0:
        logging.debug(f'0 articles scrapped!')
        break
pbar.close()

  0%|          | 0/300 [00:00<?, ?it/s]

articles to scrap: 27


  9%|▉         | 27/300 [00:14<02:30,  1.81it/s]

articles to scrap: 27


 18%|█▊        | 54/300 [00:30<02:19,  1.76it/s]

...

 90%|█████████ | 270/300 [02:32<00:16,  1.82it/s]

articles to scrap: 27


 99%|█████████▉| 297/300 [02:46<00:01,  1.85it/s]

articles to scrap: 27


100%|██████████| 300/300 [03:01<00:00,  1.65it/s]


In [9]:
for i in trange(len(scrapped_articles)):
    # print(f'{i}: {scrapped_articles[i].lead_text}')
    print(f'{i}: {len(scrapped_articles[i].text)}')

100%|██████████| 300/300 [00:00<00:00, 13610.36it/s]

0: 1493
1: 2094
2: 1549
3: 722
4: 1866
5: 1277
6: 2227
7: 1347
8: 1752
9: 2083
10: 1743
11: 1569
12: 658
13: 2375
14: 1820
15: 201
16: 1890
17: 1294
18: 2495
19: 3790
20: 2601
21: 688
22: 2630
23: 1959
24: 2767
25: 3481
26: 1467
27: 2826
28: 2648
29: 1896
30: 2850
31: 2060
32: 4205
33: 1943
34: 2246
35: 2928
36: 1679
37: 1715
38: 1744
39: 2215
40: 1858
41: 1307
42: 2122
43: 4006
44: 5512
45: 3179
46: 1630
47: 2030
48: 2131
49: 2817
50: 2631
51: 1687
52: 2319
53: 2293
54: 2003
55: 490
56: 2012
57: 2730
58: 1746
59: 1775
60: 1538
61: 3992
62: 1198
63: 1516
64: 2432
65: 3573
66: 1539
67: 4218
68: 1532
69: 4686
70: 4593
71: 1567
72: 2349
73: 2035
74: 1409
75: 1862
76: 2275
77: 1178
78: 2631
79: 1297
80: 5504
81: 3417
82: 535
83: 1843
84: 2955
85: 1642
86: 1414
87: 1268
88: 1485
89: 2634
90: 3715
91: 1967
92: 1801
93: 2137
94: 3065
95: 1735
96: 1897
97: 1115
98: 1949
99: 2069
100: 1872
101: 5452
102: 3281
103: 2018
104: 2396
105: 3103
106: 2578
107: 2287
108: 5998
109: 1408
110: 2979
111: 4

In [10]:
pr24_dataframe = pd.DataFrame(article.__dict__ for article in scrapped_articles)
pr24_dataframe

,title,title_int,lead_text,lead_text_int,link,text,when_published,author,source,portal
0,"Zjednoczona Prawica liderem, PSL poza Sejmem. ...","Zjednoczona Prawica liderem, PSL poza Sejmem. ...",Jak wynika z sondażu pracowni Social Changes p...,Jak wynika z sondażu pracowni Social Changes p...,https://www.polskieradio24.pl/5/1222/Artykul/2...,"""Koalicja Obywatelska zmniejszyła dystans dzie...",2022-01-19 22:30:00,ms,pap,pr24
1,Koronawirus w Polsce. Prof. Gut: spada liczba ...,Koronawirus w Polsce. Prof. Gut: spada liczba ...,"- Są ferie, imprezy, mamy karnawał, jest więce...","- Są ferie, imprezy, mamy karnawał, jest więce...",https://www.polskieradio24.pl/130/6257/Artykul...,Środowe dane Ministerstwa Zdrowia mówią o 30 5...,2022-01-19 22:05:00,bartos,polskieradio24,pr24
2,Obowiązek pracy zdalnej w administracji public...,Obowiązek pracy zdalnej w administracji public...,Minister zdrowia Adam Niedzielski poinformował...,Minister zdrowia Adam Niedzielski poinformował...,https://www.polskieradio24.pl/5/1222/Artykul/2...,Adam Niedzielskibył w środę gościem TVP Info. ...,2022-01-19 21:43:00,dn,iar,pr24
3,"""Potrzeba dalszych rozmów"". Premier Czech o sp...","""Potrzeba dalszych rozmów"". Premier Czech o sp...","Premier Czech Petr Fiala powiedział, że osiągn...","Premier Czech Petr Fiala powiedział, że osiągn...",https://www.polskieradio24.pl/5/3/Artykul/2887...,"Petr Fiala powiedział, że jestinformowany o po...",2022-01-19 20:55:00,kmp,iar,pr24
4,"Szef MZ mówi o ""eksplozji epidemii"" w Polsce. ...","Szef MZ mówi o ""eksplozji epidemii"" w Polsce. ...","Wg danych pandemicznych, które zostaną przedst...","Wg danych pandemicznych, które zostaną przedst...",https://www.polskieradio24.pl/5/1222/Artykul/2...,Środowe dane Ministerstwa Zdrowia mówią o 30 5...,2022-01-19 20:41:00,"ms, PAP, TVP Info",polskieradio24pl,pr24
...,...,...,...,...,...,...,...,...,...,...
295,Rząd przekazał Facebookowi sprzeciw wobec blok...,Rząd przekazał Facebookowi sprzeciw wobec blok...,Janusz Cieszyński zapowiedział kontynuację pra...,Janusz Cieszyński zapowiedział kontynuację pra...,https://www.polskieradio24.pl/5/3/Artykul/2884...,- Będziemy kontynuowali pracę zarówno w kraju ...,2022-01-14 16:29:00,"MF, PolskieRadio24.pl",polskieradio24pl,pr24
296,Tektura zamiast tylnej szyby. Tak kierowca wió...,Tektura zamiast tylnej szyby. Tak kierowca wió...,Policyjny patrol ruchu drogowego w Wiśniewie n...,Policyjny patrol ruchu drogowego w Wiśniewie n...,https://www.polskieradio24.pl/5/1222/Artykul/2...,Jak poinformowała rzeczniczkaKomendy Powiatowe...,2022-01-14 16:16:00,dn,iarpap,pr24
297,Historia sprawców zbrodni w Auschwitz. Muzeum ...,Historia sprawców zbrodni w Auschwitz. Muzeum ...,Muzeum Auschwitz udostępniło za pośrednictwem ...,Muzeum Auschwitz udostępniło za pośrednictwem ...,https://www.polskieradio24.pl/5/1222/Artykul/2...,Autorem lekcji o załodze SS jest kierownik cen...,2022-01-14 15:44:00,nj,pap,pr24
298,"""Rzeźba w przestrzeni publicznej dla Niepodleg...","""Rzeźba w przestrzeni publicznej dla Niepodleg...","Wicepremier, minister kultury i dziedzictwa na...","Wicepremier, minister kultury i dziedzictwa na...",https://www.polskieradio24.pl/5/1222/Artykul/2...,W piątek w Muzeum Rzeźby im. Xawerego Dunikows...,2022-01-14 15:41:00,nt,pap,pr24


In [11]:
timestamp = int((datetime.now(timezone.utc)).timestamp())

In [12]:
pr24_dataframe.to_parquet(f'{data_dir}/{portal_name}_{timestamp}.parquet')
pr24_dataframe.to_json(f'{data_dir}/{portal_name}_{timestamp}.json')

Krótki test - dane pierwszego artykułu

In [13]:
print(f'title:\n  {pr24_dataframe.iloc[0]["title"]}')
print(f'lead_text:\n  {pr24_dataframe.iloc[0]["lead_text"]}')
print(f'link:\n  {pr24_dataframe.iloc[0]["link"]}')
print(f'text:\n  {pr24_dataframe.iloc[0]["text"]}')
print(f'when_published:\n  {pr24_dataframe.iloc[0]["when_published"]}')

title:
  Zjednoczona Prawica liderem, PSL poza Sejmem. Sprawdź nowy sondaż
lead_text:
  Jak wynika z sondażu pracowni Social Changes przeprowadzonego na zlecenie portalu wPolityce.pl, największym poparciem cieszy się Zjednoczona Prawica (34 proc.).
                        
                          ...
link:
  https://www.polskieradio24.pl/5/1222/Artykul/2887628,Zjednoczona-Prawica-liderem-PSL-poza-Sejmem-Sprawdz-nowy-sondaz
text:
  "Koalicja Obywatelska zmniejszyła dystans dzielący ją od Zjednoczonej Prawicy. KO zyskuje kosztem Polski 2050 Szymona Hołowni. Wciąż bardzo dobry wynik notuje Konfederacja. Zyskuje również Lewica, która może się cieszyć najlepszym wynikiem od wielu tygodni" – podsumował portal, który w środę opublikował wyniki ankiety. Nowy sondaż. Na podium ZP, KO i Polska 2050 Według informacji opublikowanej przez portal, chęć głosowania naZjednoczoną Prawicęzadeklarowało 34 proc. badanych (spadek o 2 punkty procentowe w porównaniu z poprzednim sondażem). Zamiar głosowani

In [14]:
print(f'title:\n  {pr24_dataframe.iloc[99]["title"]}')
print(f'lead_text:\n  {pr24_dataframe.iloc[99]["lead_text"]}')
print(f'link:\n  {pr24_dataframe.iloc[99]["link"]}')
print(f'text:\n  {pr24_dataframe.iloc[99]["text"]}')
print(f'when_published:\n  {pr24_dataframe.iloc[99]["when_published"]}')

title:
  Pogodnie, możliwe słabe opady śniegu. Sprawdź prognozę pogody na wtorek
lead_text:
  Według prognozy we wtorek w ciągu dnia w całym kraju będzie dość pogodnie. Synoptyk Dorota Pacocha wskazała, że jedynie we wschodniej części Polski może popadać
                        
                          ...
link:
  https://www.polskieradio24.pl/5/1222/Artykul/2886834,Pogodnie-mozliwe-slabe-opady-sniegu-Sprawdz-prognoze-pogody-na-wtorek
text:
  Według prognozy przygotowanej przez Dorotę Pacochę z Instytutu Meteorologii i Gospodarki Wodnej - Państwowego Instytutu Badawczego, we wtorek w ciągu dnia będzie raczej pogodnie. Jak dodała, jedynie w Karpatach mogą wystąpićzawieje i zamiecie śnieżne.- We wtorek pogoda poprawia się. W ciągu dnia możemy spodziewać się słabych opadówśniegu, a lokalnie deszczu ze śniegiem - przekazała Dorota Pacocha. PonadtoInstytut Meteorologii i Gospodarki Wodnej wydał ostrzeżenia przed opadami marznącymi dla całej północnej części kraju. Jak powiedziała synoptyk